In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sammcmanagan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import nltk
import pandas as pd
from nltk.corpus import wordnet as wn

 

ModuleNotFoundError: No module named 'nltk'

In [323]:
# Initialize an empty list to store relationships
data = []

# Define a function to add relationships to the data list
def add_relationships(subject_synset, relation, objects):
    subject_name = subject_synset.lemma_names()[0]  # Extract human-readable name of the subject
    for obj in objects:
        data.append({
            "subject": subject_name,
            "relation": relation,
            "object": obj.lemma_names()[0],  # Extract human-readable name of the object
        })

# Define a function to add synonym relationships
def add_synonym_relationships(synset):
    subject_name = synset.lemma_names()[0]  # Use the first lemma name as the subject
    # Iterate over all the lemmas in the synset
    for lemma in synset.lemmas():
        # Add synonym relationships to the data list
        if lemma.name() != subject_name:  # Avoid self-referencing
            data.append({
                "subject": subject_name,
                "relation": "synonym",
                "object": lemma.name(),
            })

# Iterate over all synsets in WordNet
for synset in wn.all_synsets():
    subject = synset
    
    # Add synonyms
    add_synonym_relationships(synset)
    
    # Extract hypernyms
    add_relationships(synset, "hypernym", synset.hypernyms())
    #add_relationships(synset, "root_hypernym", synset.root_hypernyms())
    
    # Extract hyponyms
    #add_relationships(synset, "hyponym", synset.hyponyms())
    
    # Extract meronyms (part meronyms and substance meronyms)
    add_relationships(synset, "part_meronym", synset.part_meronyms())
    add_relationships(synset, "member_meronym", synset.member_meronyms())
    
    # Extract holonyms (part holonyms and substance holonyms)
    #add_relationships(synset, "part_holonym", synset.part_holonyms())
    #add_relationships(synset, "member_holonym", synset.member_holonyms())
    
    # Extract entailments (for verbs)
    add_relationships(synset, "entailment", synset.entailments())
    
    # Extract attributes
    add_relationships(synset, "attribute", synset.attributes())
    

    
    #coordinate_terms = get_coordinate_terms(synset)
    #add_relationships(subject, "coordinate_term", coordinate_terms)


# Convert the list of relationships to a DataFrame
df = pd.DataFrame(data)

# Display a preview of the DataFrame
print(df.head())
print(df.shape)

   subject   relation   object
0     able  attribute  ability
1     able  attribute  ability
2   unable  attribute  ability
3  abaxial    synonym   dorsal
4  adaxial    synonym  ventral
(201484, 3)


In [176]:
df.sample(n=20)

,subject,relation,object
280531,blink,root_hypernym,act
179863,United_States,part_meronym,Massachusetts
8008,earlyish,root_hypernym,earlyish
132816,reappraisal,synonym,reassessment
110254,straw,hypernym,tube
105469,rack,hypernym,framework
146306,badinage,root_hypernym,entity
144123,lettercard,hypernym,postcard
316756,condescend,root_hypernym,act
217901,Florida_yew,hypernym,yew


In [259]:
import pandas as pd
import json
import re
from tqdm import tqdm  # For progress tracking

# Function to normalize text
def normalize_text(text):
    return re.sub(r"[^\w\s]", "", text.lower()).split()

import json
from tqdm import tqdm

# Function to normalize text (assuming it is defined elsewhere)
def normalize_text(text):
    # Add text normalization logic here (e.g., lowercasing, punctuation removal, etc.)
    return text.split()  # Simple placeholder for text normalization

def load_region_descriptions(file_path):
    with open(file_path, 'r') as f:
        region_descriptions = json.load(f)
    
    # Initialize variables to store words and count the number of phrases
    all_words = set()
    word_count = 0
    phrase_count = 0
    
    # Iterate over region descriptions and count phrases
    for entry in tqdm(region_descriptions, desc="Processing region descriptions"):
        for region in entry['regions']:
            words = normalize_text(region['phrase'])
            all_words.update(words)
            word_count += len(words)
            phrase_count += 1  # Increment the phrase count
    
    return all_words, phrase_count, word_count

# Step 2: Filter WordNet DataFrame
def filter_wordnet_df(wordnet_df, valid_words):
    # Check if either subject or object is in the valid words set
    return wordnet_df[
        (wordnet_df['subject'].apply(lambda x: x in valid_words)) |
        (wordnet_df['object'].apply(lambda x: x in valid_words))
    ]

# Main function
def main(wordnet_df, region_descriptions_path, output_path):
    
    # Load the region descriptions
    valid_words, n_phrases, word_count = load_region_descriptions(region_descriptions_path)
    
    # Filter the WordNet DataFrame
    print("Filtering WordNet DataFrame...")
    filtered_df = filter_wordnet_df(wordnet_df, valid_words)
    
    # Save the filtered DataFrame
    filtered_df.to_csv(output_path, index=False)
    print(f"Filtered DataFrame saved to {output_path}")
    
    return filtered_df, n_phrases, word_count

# Paths
region_descriptions_path = "/Users/sammcmanagan/Downloads/region_descriptions.json"
output_path = "filtered_wordnet.csv"

# Run the script
filtereed_df, n_phrases, word_count = main(df, region_descriptions_path, output_path)

print(n_phrases)
print(word_count)

Processing region descriptions: 100%|██████████| 108077/108077 [00:02<00:00, 49937.76it/s]


Filtering WordNet DataFrame...
Filtered DataFrame saved to filtered_wordnet.csv
4297502
21907303


In [ ]:
df[1000:].head(10)


,subject,relation,object
1000,unattributable,root_hypernym,unattributable
1001,unattributable,attribute,attribution
1002,attributive,root_hypernym,attributive
1003,attributive_genitive,root_hypernym,attributive_genitive
1004,predicative,root_hypernym,predicative
1005,pregnant,root_hypernym,pregnant
1006,big,root_hypernym,big
1007,nonpregnant,root_hypernym,nonpregnant
1008,audible,root_hypernym,audible
1009,audible,attribute,audibility


In [352]:

print(len(df))
print(len(filtereed_df))

201484
97342


In [353]:
filtereed_df.sample(10)

,subject,relation,object
173441,fresh_water,hypernym,water
87919,submission,hypernym,agreement
1913,fresh,synonym,unused
48027,barrel,part_meronym,breech
135565,Beta,synonym,genus_Beta
56056,home,synonym,rest_home
69643,savoir-faire,synonym,address
185950,cover,synonym,plow
126354,Charles,synonym,Jacques_Alexandre_Cesar_Charles
112150,liman,hypernym,lagoon


In [354]:
df_filtered = filtereed_df[filtereed_df['subject'] != filtereed_df['object']]
print(len(df_filtered))

97101


In [317]:
import json
from collections import Counter
import re

def extract_entities_from_phrase(phrase):
    """
    This function extracts entities from the given phrase.
    We'll use a simple heuristic approach by splitting the phrase into words.
    In a more advanced setup, you could use NLP tools like spaCy for POS tagging to identify proper nouns (nouns representing entities).
    """
    # Lowercasing and basic tokenization to extract potential entities (nouns, etc.)
    phrase = phrase.lower()  # Normalize the text
    words = re.findall(r'\b\w+\b', phrase)  # Tokenize into words
    
    # Example of filtering out common stopwords; this can be improved with more sophisticated NLP methods.
    stopwords = set(['is', 'the', 'on', 'in', 'a', 'and', 'of', 'to', 'for', 'be'])
    entities = [word for word in words if word not in stopwords]
    
    return entities

# Load region descriptions JSON (replace with your actual path)
with open('/Users/sammcmanagan/Downloads/region_descriptions.json', 'r') as f:
    data = json.load(f)

# Extract entities from each phrase and count their frequencies
entity_counter = Counter()
for region_data in data:
    for region in region_data['regions']:
        entities = extract_entities_from_phrase(region['phrase'])
        entity_counter.update(entities)

# Get the most common entities
min_frequency = 300  # Set a frequency threshold (you can adjust this)
valid_entities = {entity for entity, count in entity_counter.items() if count >= min_frequency}

# Output some example valid entities
print(f"Valid entities (>= {min_frequency} occurrences):")
print(list(valid_entities)[:20])  # Print first 20 valid entities as an example

Valid entities (>= 300 occurrences):
['watermelon', 'painting', 'rail', 'goggles', 'laptops', 'container', 'pump', 'handled', 'traveling', 'left', 'decoration', 'suitcase', 'coach', 'us', 'strawberry', 'panda', 'rimmed', 'power', 'toothbrushes', 'pizzas']


In [318]:
print(len(valid_entities))

2610


In [319]:
def filter_by_visual_genome_frequency(wordnet_df, valid_entities):
    # Filter rows where the subject or object is in the valid_entities set
    return wordnet_df[
        wordnet_df['subject'].isin(valid_entities) & 
        wordnet_df['object'].isin(valid_entities)
    ]

# Apply filtering to the WordNet DataFrame (assuming you have it as `wordnet_df`)
final_df = filter_by_visual_genome_frequency(df_filtered, valid_entities)

# Show the filtered DataFrame size and first few rows
print(final_df.shape)
print(final_df.head())

(3626, 3)
    subject relation   object
98     away  synonym  outside
236   plain  synonym     bare
237   plain  synonym    spare
414    full  synonym     good
416    wide  synonym     full


In [351]:
final_df.sample(n=12)

,subject,relation,object
67270,yard,hypernym,enclosure
6103,small,attribute,size
195533,dock,hypernym,enter
55748,hat,part_meronym,crown
85451,minute,hypernym,note
171819,exhaust,hypernym,gas
86658,material,hypernym,information
87340,decoration,synonym,ribbon
53484,enclosure,hypernym,area
65599,tongs,hypernym,device


In [332]:
print(final_df[:12])

       subject relation   object
98        away  synonym  outside
236      plain  synonym     bare
237      plain  synonym    spare
414       full  synonym     good
416       wide  synonym     full
423       bare  synonym    spare
507  following  synonym     next
520       back  synonym     hind
701        big  synonym    great
702        big  synonym    large
703        big  synonym    heavy
890      hairy  synonym   haired


In [196]:
final_df.to_csv(output_path, index=False)

In [ ]:
###GRAVEYARD

# Define a function to get coordinate terms
def get_coordinate_terms(synset):
    coordinate_terms = set()
    for hypernym in synset.hypernyms():  # Get the immediate hypernyms
        # Add all hyponyms of the hypernym
        coordinate_terms.update(hypernym.hyponyms())
    # Remove the original synset to keep only the coordinate terms
    coordinate_terms.discard(synset)
    return coordinate_terms